In [14]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from catboost import Pool, CatBoostRegressor, cv
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import datetime
from tqdm import tqdm
%matplotlib inline


In [15]:
def make_submission(test, y_p, path):
    pd.DataFrame(list(zip(test.index, y_p))).to_csv(path, index=False, header=['unique_id', 'target'])

In [16]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [26]:
categorical_features = [
    'Организатор',
    'Дата начала подачи предложений',
    'Дата начала подачи предложений_month',
    'Дата начала подачи предложений_week',
    'Тип методу закупівлі',
    'Тип организатора',
     'Статус лота',
     'СтатусТендера', 
     'РегионОрганизатора',
     'Регион поставки',
     'Тендер',
     'Дата начала периода уточнений',
    'Дата начала периода уточнений_month',
    'Дата начала периода уточнений_week',
     'Дата завершения',
    'Дата завершения_month',
    'Дата завершения_week',
    'Дата завершения подачи предложений_month',
    'Дата завершения подачи предложений_week',
     'Дата завершения подачи предложений',
'Дата начала подачи предложений',
'Минимальная глубина CPV (2017 год)']

def getCatFeaturesIds(features):
    ids = []
    for i, f in enumerate(features):
        if f in categorical_features:
            ids.append(i)
        
    return ids
    

In [27]:
import pickle
with open('word.pkl', 'rb') as f:
    words_mean = pickle.load(f)

In [76]:
def add_usd(data, column):
    csv_file = pd.read_csv('~/mfdexport.csv', delimiter=';')
    csv_file['USD'] = csv_file['USD'].str.replace(',', '.')
    csv_file['UAH'] = csv_file['UAH'].str.replace(',', '.')
    csv_file['Date'] = pd.to_datetime(csv_file['Date'])
    data[column] = pd.to_datetime(data[column])
    result = data.merge(csv_file, left_on=column, right_on='Date', how='left')
    result = result.fillna(method='bfill')
    result['UAH'] = result['UAH'].astype(float)
    result['USD'] = result['USD'].astype(float)
    result['USD'] /= result['UAH']
    result.drop(['Date', 'UAH'], axis=1, inplace=True)
    result[column] = result[column].dt.strftime('%Y-%m-%d')
    return result.fillna(method='bfill')

def drop_redundant(data):
    data.drop(['Тип организатора', 'РегионОрганизатора', 'СуммаГарантииПредложенияЛота'], axis=1, inplace=True)
    return data

def split_dates(data):
    data = date_to_dummie(data, ['Дата начала подачи предложений', 'Дата завершения подачи предложений'])
    return data

def fillMissing(data):
    data.loc[data['СуммаГарантииПредложенияЛота'] == '-', "СуммаГарантииПредложенияЛота"] = 0

def preprocess_train(train_data):
#     train_data['Дата завершения'] = pd.to_datetime(train_data['Дата завершения'])
    train_clean = train_data.drop(['IDOrganizator'], axis = 1)
    train_clean['Тендер'] = train_clean['Тендер'].str.lower()
#     train_clean = split_dates(train_clean)
#     train_clean = drop_redundant(train_clean)

#     train_clean = train_clean[(train_clean['Статус лота'] != 'Неуспішний лот закупівлі') & (train_clean['Статус лота'] != 'Скасовано лот закупівлі')]
    return train_clean

def preprocess_test(test_data):
    test_clean = test_data.drop(['IDOrganizator'], axis = 1) 
#     test_clean = split_dates(test_clean)
#     test_clean = drop_redundant(test_clean)
    return test_clean

def extract_target(train_data):
    ids =  train_data['target']
    train_data.drop(['target'], axis = 1, inplace=True)
    return ids

In [77]:
def split_data(train_data):
    total = len(train_data)
    train_split = train_data.sort_values('Дата начала подачи предложений').iloc[:int(total * 0.7)]
    test_split = train_data.sort_values('Дата начала подачи предложений').iloc[int(total * 0.7):]
    return train_split, test_split

In [92]:
# Read data
train_data = pd.read_csv('data/train_agg_3.csv', index_col = 0)
test_data = pd.read_csv('data/test.csv', index_col = 0)
tmp = test_data.copy()
train_data = add_usd(train_data, 'Дата начала подачи предложений')
test_data = add_usd(test_data, 'Дата начала подачи предложений')

fillMissing(train_data)
fillMissing(test_data)

In [93]:
train_p = preprocess_train(train_data)
test_p = preprocess_test(test_data)

# make split
# train_p, test_p = split_data(train_p)
train_labels = extract_target(train_p)
# test_labels = extract_target(test_p)
cats = getCatFeaturesIds(list(train_p.columns))

In [94]:
train_pool = Pool(train_p, train_labels, cat_features=cats)
test_pool = Pool(test_p, cat_features=cats)

In [83]:
model = CatBoostRegressor(iterations=6000, rsm=0.7, depth=4, leaf_estimation_method='Gradient', learning_rate=0.2, loss_function="RMSE")

In [84]:
model.fit(train_pool, plot=True)

0:	learn: 2.0038686	total: 360ms	remaining: 35m 56s


CatboostIpythonWidget(data={'.': {'path': '.', 'name': 'current', 'content': {'passed_iterations': 1, 'total_iterations': 6000, 'rows': {'learn_error': [['iter', 'RMSE'], ['0', '2.00386864']], 'test_error': [], 'time_left': [['iter', 'Passed', 'Remaining'], ['0', '359', '2156847']], 'meta': {'name': 'experiment', 'iterCount': '6000', 'learnErrorLog': 'learn_error.tsv', 'testErrorLog': 'test_error.tsv', 'timeLeft': 'time_left.tsv', 'loss_RMSE': 'min'}}}}}, layout=Layout(align_self='stretch', height='500px'))

1:	learn: 1.7621815	total: 539ms	remaining: 26m 57s
2:	learn: 1.5880835	total: 673ms	remaining: 22m 26s
3:	learn: 1.4654482	total: 912ms	remaining: 22m 47s
4:	learn: 1.3786640	total: 1.09s	remaining: 21m 45s
5:	learn: 1.3180595	total: 1.38s	remaining: 23m 2s
6:	learn: 1.2789773	total: 1.67s	remaining: 23m 49s
7:	learn: 1.2455706	total: 2.04s	remaining: 25m 27s
8:	learn: 1.2256808	total: 2.34s	remaining: 25m 56s
9:	learn: 1.2084228	total: 2.66s	remaining: 26m 31s
10:	learn: 1.1994275	total: 2.96s	remaining: 26m 53s
11:	learn: 1.1930668	total: 3.29s	remaining: 27m 22s
12:	learn: 1.1888089	total: 3.53s	remaining: 27m 4s
13:	learn: 1.1855667	total: 3.82s	remaining: 27m 14s
14:	learn: 1.1819934	total: 4.11s	remaining: 27m 21s
15:	learn: 1.1797143	total: 4.38s	remaining: 27m 17s
16:	learn: 1.1736162	total: 4.58s	remaining: 26m 53s
17:	learn: 1.1712630	total: 4.96s	remaining: 27m 29s
18:	learn: 1.1692761	total: 5.17s	remaining: 27m 7s
19:	learn: 1.1678072	total: 5.38s	remaining: 26m 48s
20:	l

157:	learn: 1.1105302	total: 41.5s	remaining: 25m 33s
158:	learn: 1.1104420	total: 41.7s	remaining: 25m 32s
159:	learn: 1.1102718	total: 42s	remaining: 25m 33s
160:	learn: 1.1101662	total: 42.3s	remaining: 25m 33s
161:	learn: 1.1101055	total: 42.6s	remaining: 25m 34s
162:	learn: 1.1100972	total: 42.8s	remaining: 25m 34s
163:	learn: 1.1100243	total: 43.1s	remaining: 25m 35s
164:	learn: 1.1099184	total: 43.4s	remaining: 25m 34s
165:	learn: 1.1098623	total: 43.6s	remaining: 25m 33s
166:	learn: 1.1097928	total: 43.8s	remaining: 25m 31s
167:	learn: 1.1096931	total: 44.1s	remaining: 25m 30s
168:	learn: 1.1094414	total: 44.4s	remaining: 25m 30s
169:	learn: 1.1094196	total: 44.6s	remaining: 25m 28s
170:	learn: 1.1093428	total: 44.8s	remaining: 25m 28s
171:	learn: 1.1092485	total: 45.1s	remaining: 25m 29s
172:	learn: 1.1091587	total: 45.4s	remaining: 25m 28s
173:	learn: 1.1090982	total: 45.7s	remaining: 25m 30s
174:	learn: 1.1090375	total: 45.9s	remaining: 25m 28s
175:	learn: 1.1089672	total: 4

309:	learn: 1.1009931	total: 1m 20s	remaining: 24m 45s
310:	learn: 1.1009911	total: 1m 21s	remaining: 24m 44s
311:	learn: 1.1009466	total: 1m 21s	remaining: 24m 43s
312:	learn: 1.1009267	total: 1m 21s	remaining: 24m 44s
313:	learn: 1.1009094	total: 1m 21s	remaining: 24m 43s
314:	learn: 1.1008812	total: 1m 22s	remaining: 24m 42s
315:	learn: 1.1008480	total: 1m 22s	remaining: 24m 41s
316:	learn: 1.1008158	total: 1m 22s	remaining: 24m 42s
317:	learn: 1.1007971	total: 1m 22s	remaining: 24m 41s
318:	learn: 1.1007233	total: 1m 23s	remaining: 24m 41s
319:	learn: 1.1006456	total: 1m 23s	remaining: 24m 40s
320:	learn: 1.1006053	total: 1m 23s	remaining: 24m 41s
321:	learn: 1.1005961	total: 1m 23s	remaining: 24m 40s
322:	learn: 1.1005068	total: 1m 24s	remaining: 24m 40s
323:	learn: 1.1004670	total: 1m 24s	remaining: 24m 38s
324:	learn: 1.1004485	total: 1m 24s	remaining: 24m 36s
325:	learn: 1.1004360	total: 1m 24s	remaining: 24m 35s
326:	learn: 1.1004179	total: 1m 25s	remaining: 24m 34s
327:	learn

...

5324:	learn: 1.0508213	total: 24m 23s	remaining: 3m 5s
5325:	learn: 1.0508110	total: 24m 23s	remaining: 3m 5s
5326:	learn: 1.0508002	total: 24m 23s	remaining: 3m 4s
5327:	learn: 1.0507982	total: 24m 24s	remaining: 3m 4s
5328:	learn: 1.0507972	total: 24m 24s	remaining: 3m 4s
5329:	learn: 1.0507869	total: 24m 24s	remaining: 3m 4s
5330:	learn: 1.0507848	total: 24m 25s	remaining: 3m 3s
5331:	learn: 1.0507833	total: 24m 25s	remaining: 3m 3s
5332:	learn: 1.0507819	total: 24m 25s	remaining: 3m 3s
5333:	learn: 1.0507777	total: 24m 25s	remaining: 3m 3s
5334:	learn: 1.0507706	total: 24m 26s	remaining: 3m 2s
5335:	learn: 1.0507670	total: 24m 26s	remaining: 3m 2s
5336:	learn: 1.0507660	total: 24m 26s	remaining: 3m 2s
5337:	learn: 1.0507626	total: 24m 27s	remaining: 3m 1s
5338:	learn: 1.0507571	total: 24m 27s	remaining: 3m 1s
5339:	learn: 1.0507498	total: 24m 27s	remaining: 3m 1s
5340:	learn: 1.0507447	total: 24m 28s	remaining: 3m 1s
5341:	learn: 1.0507410	total: 24m 28s	remaining: 3m
5342:	learn: 

5472:	learn: 1.0499602	total: 25m 5s	remaining: 2m 24s
5473:	learn: 1.0499565	total: 25m 5s	remaining: 2m 24s
5474:	learn: 1.0499522	total: 25m 6s	remaining: 2m 24s
5475:	learn: 1.0499491	total: 25m 6s	remaining: 2m 24s
5476:	learn: 1.0499368	total: 25m 6s	remaining: 2m 23s
5477:	learn: 1.0499346	total: 25m 7s	remaining: 2m 23s
5478:	learn: 1.0499315	total: 25m 7s	remaining: 2m 23s
5479:	learn: 1.0499290	total: 25m 7s	remaining: 2m 23s
5480:	learn: 1.0499262	total: 25m 7s	remaining: 2m 22s
5481:	learn: 1.0499205	total: 25m 8s	remaining: 2m 22s
5482:	learn: 1.0499156	total: 25m 8s	remaining: 2m 22s
5483:	learn: 1.0499109	total: 25m 8s	remaining: 2m 21s
5484:	learn: 1.0499076	total: 25m 8s	remaining: 2m 21s
5485:	learn: 1.0499036	total: 25m 9s	remaining: 2m 21s
5486:	learn: 1.0499011	total: 25m 9s	remaining: 2m 21s
5487:	learn: 1.0498942	total: 25m 9s	remaining: 2m 20s
5488:	learn: 1.0498852	total: 25m 10s	remaining: 2m 20s
5489:	learn: 1.0498679	total: 25m 10s	remaining: 2m 20s
5490:	le

5620:	learn: 1.0492184	total: 25m 48s	remaining: 1m 44s
5621:	learn: 1.0492075	total: 25m 48s	remaining: 1m 44s
5622:	learn: 1.0492067	total: 25m 48s	remaining: 1m 43s
5623:	learn: 1.0492024	total: 25m 48s	remaining: 1m 43s
5624:	learn: 1.0491995	total: 25m 49s	remaining: 1m 43s
5625:	learn: 1.0491950	total: 25m 49s	remaining: 1m 43s
5626:	learn: 1.0491894	total: 25m 49s	remaining: 1m 42s
5627:	learn: 1.0491882	total: 25m 50s	remaining: 1m 42s
5628:	learn: 1.0491862	total: 25m 50s	remaining: 1m 42s
5629:	learn: 1.0491845	total: 25m 50s	remaining: 1m 41s
5630:	learn: 1.0491793	total: 25m 50s	remaining: 1m 41s
5631:	learn: 1.0491778	total: 25m 51s	remaining: 1m 41s
5632:	learn: 1.0491775	total: 25m 51s	remaining: 1m 41s
5633:	learn: 1.0491762	total: 25m 51s	remaining: 1m 40s
5634:	learn: 1.0491732	total: 25m 52s	remaining: 1m 40s
5635:	learn: 1.0491712	total: 25m 52s	remaining: 1m 40s
5636:	learn: 1.0491698	total: 25m 52s	remaining: 1m 39s
5637:	learn: 1.0491672	total: 25m 52s	remaining:

5768:	learn: 1.0484023	total: 26m 29s	remaining: 1m 3s
5769:	learn: 1.0483898	total: 26m 29s	remaining: 1m 3s
5770:	learn: 1.0483828	total: 26m 30s	remaining: 1m 3s
5771:	learn: 1.0483808	total: 26m 30s	remaining: 1m 2s
5772:	learn: 1.0483772	total: 26m 30s	remaining: 1m 2s
5773:	learn: 1.0483614	total: 26m 30s	remaining: 1m 2s
5774:	learn: 1.0483586	total: 26m 31s	remaining: 1m 1s
5775:	learn: 1.0483555	total: 26m 31s	remaining: 1m 1s
5776:	learn: 1.0483532	total: 26m 31s	remaining: 1m 1s
5777:	learn: 1.0483505	total: 26m 32s	remaining: 1m 1s
5778:	learn: 1.0483468	total: 26m 32s	remaining: 1m
5779:	learn: 1.0483436	total: 26m 32s	remaining: 1m
5780:	learn: 1.0483418	total: 26m 32s	remaining: 1m
5781:	learn: 1.0483407	total: 26m 33s	remaining: 1m
5782:	learn: 1.0483388	total: 26m 33s	remaining: 59.8s
5783:	learn: 1.0483362	total: 26m 33s	remaining: 59.5s
5784:	learn: 1.0483334	total: 26m 33s	remaining: 59.2s
5785:	learn: 1.0483257	total: 26m 34s	remaining: 59s
5786:	learn: 1.0483239	t

5920:	learn: 1.0476683	total: 27m 11s	remaining: 21.8s
5921:	learn: 1.0476447	total: 27m 11s	remaining: 21.5s
5922:	learn: 1.0476381	total: 27m 11s	remaining: 21.2s
5923:	learn: 1.0476350	total: 27m 12s	remaining: 20.9s
5924:	learn: 1.0476269	total: 27m 12s	remaining: 20.7s
5925:	learn: 1.0476223	total: 27m 12s	remaining: 20.4s
5926:	learn: 1.0476185	total: 27m 13s	remaining: 20.1s
5927:	learn: 1.0476111	total: 27m 13s	remaining: 19.8s
5928:	learn: 1.0476069	total: 27m 13s	remaining: 19.6s
5929:	learn: 1.0475995	total: 27m 13s	remaining: 19.3s
5930:	learn: 1.0475964	total: 27m 14s	remaining: 19s
5931:	learn: 1.0475950	total: 27m 14s	remaining: 18.7s
5932:	learn: 1.0475864	total: 27m 14s	remaining: 18.5s
5933:	learn: 1.0475689	total: 27m 15s	remaining: 18.2s
5934:	learn: 1.0475567	total: 27m 15s	remaining: 17.9s
5935:	learn: 1.0475533	total: 27m 15s	remaining: 17.6s
5936:	learn: 1.0475510	total: 27m 16s	remaining: 17.4s
5937:	learn: 1.0475499	total: 27m 16s	remaining: 17.1s
5938:	learn:

In [85]:
list(zip(train_p.columns, model.feature_importances_))

[('Минимальная глубина CPV (2017 год)', 14.941586347140651),
 ('СуммаТендера', 3.3394290658388295),
 ('Организатор', 7.801874526142127),
 ('Дата начала подачи предложений', 2.119595134052435),
 ('Тип методу закупівлі', 8.33369809121122),
 ('Тип организатора', 0.8967416121613786),
 ('Статус лота', 20.292539761296894),
 ('СтатусТендера', 11.543804053118615),
 ('РегионОрганизатора', 1.4771693604750393),
 ('Регион поставки', 1.3736515528256439),
 ('Тендер', 4.599205939638756),
 ('Дата начала периода уточнений', 3.6744730306853595),
 ('Дата завершения', 7.1248939207110205),
 ('Дата завершения подачи предложений', 4.971444525029665),
 ('ШагУменьшенияЛот', 4.460590730649617),
 ('СуммаГарантииПредложенияЛота', 1.1044053225916766),
 ('USD', 1.9448970264310712)]

In [86]:
y_p = model.predict(test_pool)

In [95]:
y_p[y_p < 1] = 1

In [96]:
test_p.index = tmp.index

In [97]:
y_p

array([2.07757037, 2.84062319, 3.18329505, ..., 1.34881147, 2.51021405,
       2.43283479])

In [ ]:
rmsle(test_labels, y_p)

In [98]:
make_submission(test_p, y_p, '~/results/submission_1657.csv')

In [ ]:
total = len(train_p)
train_p.sort_values('Дата начала подачи предложений').iloc[int(total * 0.7):]

In [ ]:
import pickle
with open('word.pkl', 'rb') as f:
    words_mean = pickle.load(f)

In [ ]:
words_mean

In [ ]:
train_p['Тендер'] = train_p['Тендер'].str.lower()

In [9]:
train_data['word_weight'] = 0
for word in tqdm(words_mean.keys(), total=len(words_mean)):
    train_data.loc[train_data['Тендер'].str.contains(word), 'word_weight'] += words_mean[word]

100%|██████████| 1000/1000 [11:30<00:00,  1.45it/s]


In [10]:
test_data['word_weight'] = 0
for word in tqdm(words_mean.keys(), total=len(words_mean)):
    test_data.loc[test_data['Тендер'].str.contains(word), 'word_weight'] += words_mean[word]

100%|██████████| 1000/1000 [00:59<00:00, 16.71it/s]


In [ ]:
train_p

In [ ]:
list(zip(test_p.word_weight.values, test_labels))

In [ ]:
list(zip(train_p.columns, model.feature_importances_))

In [20]:
for x in train_data['Тендер']:
    if 'икона' in x:
        print(x)

Виконання робіт по об’єкту: «Капітальний ремонт будівлі та благоустрій території КЗО  «СЗШ №34» ДМР за адресою: м. Дніпро, вул. Д. Кедрина, 57»
ДК 021:2015 71248000-8 Технічний нагляд за проектами та документацією (Виконання технічного нагляду по об’єкту: 45454100-5 Відновлювальні роботи(« Реконструкція проспекту Миру від вул. Пушкіна до вул.Побєди у м. Курахове»( 1 черга).)
ДК 021:2015 71248000-8 Технічний нагляд за проектами та документацією (Виконання технічного нагляду по об’єкту: 45454100-5 Відновлювальні роботи(« Реконструкція проспекту Миру від вул. Пушкіна до вул.Побєди у м. Курахове»( 1 черга).)
ДК 021:2015 71248000-8 Технічний нагляд за проектами та документацією (Виконання технічного нагляду по об’єкту: 45454100-5 Відновлювальні роботи(« Реконструкція проспекту Миру від вул. Пушкіна до вул.Побєди у м. Курахове»( 1 черга).)
Виконання робіт з заміни суцільного стрічкового дерев'яного засклення на металопластикове
Виконання робіт з заміни суцільного стрічкового дерев'яного заск

Послуга з ремонту технічного обладнання теплового пункту з мережами, що знаходяться на балансі Департаменту охорони здоров’я виконавчого органу Київської міської ради (Київської міської державної адміністрації), за адресою: вул. Прорізна, 19, м. Київ
проведення технічного нагляду за виконанням робіт з реконструкції  ДНЗ№585 за адресою: вул.Малиновського,1-Б  в Оболонському районі м. Києва - код закупівлі згідно з класифікатором CPV: 71247000-1
проведення технічного нагляду за виконанням робіт з реконструкції  ДНЗ№585 за адресою: вул.Малиновського,1-Б  в Оболонському районі м. Києва - код закупівлі згідно з класифікатором CPV: 71247000-1
проведення технічного нагляду за виконанням робіт з реконструкції  ДНЗ№585 за адресою: вул.Малиновського,1-Б  в Оболонському районі м. Києва - код закупівлі згідно з класифікатором CPV: 71247000-1
проведення технічного нагляду за виконанням робіт з реконструкції  ДНЗ№585 за адресою: вул.Малиновського,1-Б  в Оболонському районі м. Києва - код закупівлі з

Технічний нагляд під час виконання робіт по об'єкту: "Реконструкція ФАП Новоселівка-1 за адресою: Донецька область, Ясинуватський район, с.Новоселівка, вул.Шкільна"
Технічний нагляд під час виконання робіт по об'єкту: "Реконструкція ФАП Новоселівка-1 за адресою: Донецька область, Ясинуватський район, с.Новоселівка, вул.Шкільна"
проведення технічного нагляду за виконанням робіт по реконструкції школи-інтернату №299, вул.Макіївська, 9 в Оболонському районі міста Києва.
ДСТУ Б Д.1.1-7:2013 (71322000-1 - Виконання проектних робіт з «Будівництва дренажної системи водовідведення від покрівлі паркінгу та облаштування системи поливу газонів навколо будівлі Державного підприємства обслуговування повітряного руху України в м.Бориспіль, Аеропорт»)
Код ДК 021:2015 - 45520000-8 – прокат обладнання з оператором для виконання земляних робіт 	 ( Послуги   щодо  прокату обладнання з оператором  для виконання земляних робіт  )
Код ДК 021:2015 - 45520000-8 – прокат обладнання з оператором для виконання з

Виконання технічного нагляду під час капітального ремонту спортивних майданчиків школи І-ІІІ ступенів №278 Деснянського району міста Києва
Екологічне обгрунтування виконання оздоровчих заходів р.Стрижень за межами м.Чернігова
Капітальний ремонт ВПЗ Гульськ за адресою: с. Гульськ вул. Леніна, 7 ( проходження державної експертизи з отриманням експертного звіту, виконання будівельно-монтажних робіт)
Капітальний ремонт ВПЗ Гульськ за адресою: с. Гульськ вул. Леніна, 7 ( проходження державної експертизи з отриманням експертного звіту, виконання будівельно-монтажних робіт)
Капітальний ремонт ВПЗ Гульськ за адресою: с. Гульськ вул. Леніна, 7 ( проходження державної експертизи з отриманням експертного звіту, виконання будівельно-монтажних робіт)
Виконання БМР та пусконалагоджувальних робіт з «Оснащення інженерних вводів житлових будинків комунальної форми власності, житлово-будівельних кооперативів та об’єднань співвласників багатоквартирних будинків теплолічильниками разом з програмно-апаратн